<a href="https://colab.research.google.com/github/plaban1981/DEEP-LEARNING-/blob/master/keras_auto_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic hyperparameter tuning with Keras Tuner and Tensorflow 2.0 to boost accuracy on a computer vision problem.

https://www.sicara.ai/blog/hyperparameter-tuning-keras-tuner

In [0]:
pip install keras-tuner

# Import Libraries

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
print(tf.__version__)

# Load Data

In [0]:
(train_images,train_labels),(test_images,test_labels) =fashion_mnist.load_data()

# Preprocess- Rescale pixels

In [0]:
train_images = train_images /255.0
test_images = test_images / 255.0

In [0]:
train_images.shape

# Reshape images

In [0]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
train_images.shape

In [0]:
train_images[0].shape

# Build Model - Hyperparameters

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

# Random Search to find best hyperparameters

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

# Hyperparameter tuning

In [0]:
tuner_search.search(train_images,train_labels,epochs=10,validation_split=0.1)

# Best Hyperparameter evaluated

In [0]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

# Retraining images considering the best model hyperparameters

In [0]:
history = model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

# Make Predictions

In [0]:
predictions= model.predict(test_images)

# Visualize the predictions

In [0]:
import matplotlib.pyplot as plt
import numpy as np
plt.imshow(test_images[0].reshape(28,28))
print('actual label :',test_labels[0])
print('predicted label :',np.argmax(predictions[0]))

In [0]:
plt.imshow(test_images[10].reshape(28,28))
print('actual label :',test_labels[10])
print('predicted label :',np.argmax(predictions[10]))